In [48]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
from astropy.io import ascii
%pylab inline
import matplotlib
import matplotlib.colors as colors
import sklearn
from sklearn.mixture import GaussianMixture
from sklearn import mixture   

Populating the interactive namespace from numpy and matplotlib


In [49]:
#sns.reset_defaults()
sns.set_context("notebook", font_scale=1.0, rc={"lines.linewidth": 2.5})
sns.set_palette(sns.color_palette(["#9b59b6", 
                                   "#e74c3c", "#3498db", 
                                   "#34495e", "#2ecc71"]))
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'

In [50]:
# Pandas reads from *lots* of different data sources
#data = pd.read_csv('../data/galaxy_all.csv', sep= ',') #MJ path
data = pd.read_csv('galaxy_all.csv', sep= ',') #TUGBA path

In [51]:
data = data[(data['sm']<10**11.4)&(data['sm']>10**9)&(data['sfr']>0)]
data[:0]

,GalaxyID,GroupID,SnapNum,Redshift,CentreOfMass_x,CentreOfMass_y,CentreOfMass_z,GroupNumber,SubGroupNumber,MassType_Star,...,StellarVelDisp,Vmax,sm,sfr,vel_disp,size,u_nodust,g_nodust,r_nodust,i_nodust


In [52]:
data = data.values

In [53]:
u_r = data[:,-3] - data[:,-2]
r = data[:,-2]
ssfr = np.log10(data[:,-7]/data[:,-8]) 
ms = np.log10(data[:,-8])
data[:,-8] = ms.copy()
galaxy_id = data[:,0]

In [54]:
red_id , blue_id = [] , [] 
K = 40
bins = np.linspace(9, 11.4, K)

In [56]:
for k in range(len(bins)-1):
    
    logm_min, logm_max  = bins[k], bins[k+1]
    mask = (ms>logm_min)&(ms<logm_max)
    urm = u_r[mask]
    rm = r[mask]
    gid = galaxy_id[mask]
    
    gmm = GaussianMixture(n_components = 2, n_iter=100).fit(urm.reshape(len(urm), 1))
    gmm.fit(urm.reshape(len(urm), 1))
    means_ = gmm.means_
    vars_ = gmm.covars_
    
    mu0, mu1 = means_[0][0], means_[1][0]
    std0, std1 = vars_[0][0]**.5, vars_[1][0]**.5
    
    if mu0>mu1:
        mured, mublue = mu0, mu1
        stdred , stdblue = std0, std1
    else:
        mured, mublue = mu1, mu0
        stdred , stdblue = std1, std0
        
    redmask = urm > mured - 0.5 * stdred
    bluemask = urm < mublue + 0.5 * stdblue
    red_id.append(gid[redmask])
    blue_id.append(gid[bluemask])

TypeError: __init__() got an unexpected keyword argument 'n_iter'